In [18]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score

In [2]:
matches = pd.read_csv('2016matches.csv', index_col=0)
matches2 = pd.read_csv('2021matches.csv', index_col=0)

In [13]:
matches.head()

,Date,Group/Semi Final/Final,Match Between,Venue,Winning Team Score,Losing Team Score,Winner Team,Result,Player Of The Match,Team_A,Team_B,Team_A_Code,Team_B_Code
Match No,,,,,,,,,,,,,
1,2016-08-03,Group B,Hong Kong vs Zimbabwe,"Vidarbha Cricket Association Stadium, Nagpur",158/8(20 overs),144/6(20 overs),Zimbabwe,Zimbabwe won by 14 runs,Vusi Sibanda,Hong Kong,Zimbabwe,4,17
2,2016-08-03,Group B,Afghanistan vs Scotland,"Vidarbha Cricket Association Stadium, Nagpur",170/5(20 overs),156/5(20 overs),Afghanistan,Afghanistan won by 14 runs,Mohammad Shahzad,Afghanistan,Scotland,0,13
3,2016-09-03,Group A,Bangladesh vs Netherlands,"Himachal Pradesh Cricket Association Stadium, ...",153/7(20 overs),145/7(20 overs),Bangladesh,Bangladesh won by 8 runs,Tamim Iqbal,Bangladesh,Netherlands,2,8
4,2016-09-03,Group A,Ireland vs Oman,"Himachal Pradesh Cricket Association Stadium, ...",157/8(19.4 overs),154/5(20 overs),Oman,Oman won by 2 wickets,Amir Ali,Ireland,Oman,6,10
5,2016-10-03,Group B,Scotland vs Zimbabwe,"Vidarbha Cricket Association Stadium, Nagpur",147/7(20 overs),136(19.4 overs),Zimbabwe,Zimbabwe won by 11 runs,Wellington Masakadza,Scotland,Zimbabwe,13,17
6,2016-10-03,Group B,Afghanistan vs Hong Kong,"Vidarbha Cricket Association Stadium, Nagpur",119/4(18 overs),116/6(20 overs),Afghanistan,Afghanistan won by 6 wickets,Mohammad Nabi,Afghanistan,Hong Kong,0,4
7,2016-11-03,Group A,Oman vs Netherlands,"Himachal Pradesh Cricket Association Stadium, ...",NaN,NaN,NaN,Match abandoned without a ball being bowled (w...,NaN,Oman,Netherlands,10,8
8,2016-11-03,Group A,Bangladesh vs Ireland,"Himachal Pradesh Cricket Association Stadium, ...",94/2(8 overs),NaN,NaN,No result,NaN,Bangladesh,Ireland,2,6
9,2016-12-03,Group B,Afghanistan vs Zimbabwe,"Vidarbha Cricket Association Stadium, Nagpur",186/6(20 overs),127(19.4 overs),Afghanistan,Afghanistan won by 59 runs,Mohammad Nabi,Afghanistan,Zimbabwe,0,17


In [14]:
matches.shape

(35, 9)

In [7]:
matches.dtypes

,0
Date,datetime64[ns]
Group/Semi Final/Final,object
Match Between,object
Venue,object
Winning Team Score,object
Losing Team Score,object
Winner Team,object
Result,object
Player Of The Match,object
Team_A,object


In [5]:
matches["Date"] = pd.to_datetime(matches["Date"])

In [10]:
matches["Match Between"] = matches['Match Between'].replace("SCotland vs Zimbabwe", "Scotland vs Zimbabwe")
matches2["Match Between"] = matches2['Match Between'].replace('Ireland vs Netherands', "Ireland vs Netherlands")

def preprocess_data(df):
  df[["Team_A", "Team_B"]] = df['Match Between'].str.split(" vs ", expand=True)
  df['Date'] = pd.to_datetime(df['Date'])

  return df

matches = preprocess_data(matches)
matches2 = preprocess_data(matches2)


In [11]:
all_teams = pd.concat([matches['Team_A'], matches['Team_B'], matches2['Team_A'], matches2['Team_B']]).unique()

le = LabelEncoder()
le.fit(all_teams)

matches['Team_A_Code'] = le.transform(matches['Team_A'])
matches['Team_B_Code'] = le.transform(matches['Team_B'])

matches2['Team_A_Code'] = le.transform(matches2['Team_A'])
matches2['Team_B_Code'] = le.transform(matches2['Team_B'])

print(le.classes_)

['Afghanistan' 'Australia' 'Bangladesh' 'England' 'Hong Kong' 'India'
 'Ireland' 'Namibia' 'Netherlands' 'New Zealand' 'Oman' 'Pakistan'
 'Papua New Guinea' 'Scotland' 'South Africa' 'Sri Lanka' 'West Indies'
 'Zimbabwe']


In [24]:
matches2.head()

,Date,Group/Semi Final/Final,Match Between,Venue,Winning Team Score,Losing Team Score,Winner Team,Result,Player Of The Match,Team_A,Team_B,Team_A_Code,Team_B_Code,Team_A_Target,Team_B_Target
Match No,,,,,,,,,,,,,,,
1,2021-10-17,Group B,Oman vs Papua New Guinea,"Al Amerat Cricket Ground (Ministry Turf 1), Al...",131/0(13.4 overs),129/9(20 overs),Oman,Oman won by 10 wickets,Zeeshan Maqsood,Oman,Papua New Guinea,10,12,1,0
2,2021-10-17,Group B,Bangladesh vs Scotland,"Al Amerat Cricket Ground (Ministry Turf 1), Al...",140/9(20 overs),134/7(20 overs),Scotland,Scotland won by 6 runs,Chris Greaves,Bangladesh,Scotland,2,13,0,1
3,2021-10-18,Group A,Ireland vs Netherlands,"Sheikh Zayed Stadium, Abu Dhabi",107/3(15.1 overs),106(20 overs),Ireland,Ireland won by 7 wickets,Curtis Campher,Ireland,Netherlands,6,8,1,0
4,2021-10-18,Group A,Sri Lanka vs Namibia,"Sheikh Zayed Stadium, Abu Dhabi",100/3(13.3 overs),96(19.3 overs),Sri Lanka,Sri Lanka won by 7 wickets,Maheesh Theekshana,Sri Lanka,Namibia,15,7,1,0
5,2021-10-19,Group B,Scotland vs Papua New Guinea,"Al Amerat Cricket Ground (Ministry Turf 1), Al...",165/9(20 overs),148(19.3 overs),Scotland,Scotland won by 17 runs,Richie Berrington,Scotland,Papua New Guinea,13,12,1,0


In [14]:
matches["Team_A_Target"] = matches.apply(lambda row: 1 if row["Team_A"] == row["Winner Team"] else 0, axis=1)
matches["Team_B_Target"] = matches.apply(lambda row: 1 if row["Team_B"] == row["Winner Team"] else 0, axis=1)

matches2["Team_A_Target"] = matches2.apply(lambda row: 1 if row["Team_A"] == row["Winner Team"] else 0, axis=1)
matches2["Team_B_Target"] = matches2.apply(lambda row: 1 if row["Team_B"] == row["Winner Team"] else 0, axis=1)

In [17]:
matches.head(10)

,Date,Group/Semi Final/Final,Match Between,Venue,Winning Team Score,Losing Team Score,Winner Team,Result,Player Of The Match,Team_A,Team_B,Team_A_Code,Team_B_Code,Team_A_Target,Team_B_Target
Match No,,,,,,,,,,,,,,,
1,2016-08-03,Group B,Hong Kong vs Zimbabwe,"Vidarbha Cricket Association Stadium, Nagpur",158/8(20 overs),144/6(20 overs),Zimbabwe,Zimbabwe won by 14 runs,Vusi Sibanda,Hong Kong,Zimbabwe,4,17,0,1
2,2016-08-03,Group B,Afghanistan vs Scotland,"Vidarbha Cricket Association Stadium, Nagpur",170/5(20 overs),156/5(20 overs),Afghanistan,Afghanistan won by 14 runs,Mohammad Shahzad,Afghanistan,Scotland,0,13,1,0
3,2016-09-03,Group A,Bangladesh vs Netherlands,"Himachal Pradesh Cricket Association Stadium, ...",153/7(20 overs),145/7(20 overs),Bangladesh,Bangladesh won by 8 runs,Tamim Iqbal,Bangladesh,Netherlands,2,8,1,0
4,2016-09-03,Group A,Ireland vs Oman,"Himachal Pradesh Cricket Association Stadium, ...",157/8(19.4 overs),154/5(20 overs),Oman,Oman won by 2 wickets,Amir Ali,Ireland,Oman,6,10,0,1
5,2016-10-03,Group B,Scotland vs Zimbabwe,"Vidarbha Cricket Association Stadium, Nagpur",147/7(20 overs),136(19.4 overs),Zimbabwe,Zimbabwe won by 11 runs,Wellington Masakadza,Scotland,Zimbabwe,13,17,0,1
6,2016-10-03,Group B,Afghanistan vs Hong Kong,"Vidarbha Cricket Association Stadium, Nagpur",119/4(18 overs),116/6(20 overs),Afghanistan,Afghanistan won by 6 wickets,Mohammad Nabi,Afghanistan,Hong Kong,0,4,1,0
7,2016-11-03,Group A,Oman vs Netherlands,"Himachal Pradesh Cricket Association Stadium, ...",NaN,NaN,NaN,Match abandoned without a ball being bowled (w...,NaN,Oman,Netherlands,10,8,0,0
8,2016-11-03,Group A,Bangladesh vs Ireland,"Himachal Pradesh Cricket Association Stadium, ...",94/2(8 overs),NaN,NaN,No result,NaN,Bangladesh,Ireland,2,6,0,0
9,2016-12-03,Group B,Afghanistan vs Zimbabwe,"Vidarbha Cricket Association Stadium, Nagpur",186/6(20 overs),127(19.4 overs),Afghanistan,Afghanistan won by 59 runs,Mohammad Nabi,Afghanistan,Zimbabwe,0,17,1,0


In [ ]:
matches2["Date"] = pd.to_datetime(matches2['Date'])

In [23]:
matches2.dtypes

,0
Date,datetime64[ns]
Group/Semi Final/Final,object
Match Between,object
Venue,object
Winning Team Score,object
Losing Team Score,object
Winner Team,object
Result,object
Player Of The Match,object
Team_A,object


In [19]:
rf = RandomForestClassifier()

In [20]:
x_train = matches[['Team_A_Code', 'Team_B_Code']]
y_train = matches['Team_A_Target']

x_test = matches2[['Team_A_Code', 'Team_B_Code']]
y_test = matches2['Team_A_Target']

In [21]:
rf.fit(x_train, y_train)

RandomForestClassifier()

In [22]:
y_pred = rf.predict(x_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

Accuracy: 0.51
